In [1]:
import glob
import importlib.util
import os

import geopandas as gpd
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd
import rasterio
import rioxarray as rio
import xarray as xr
from IPython.display import display
from osgeo import gdal
from rasterio.enums import Resampling

plt.style.use("ggplot")

In [2]:
# Import CL functions
spec = importlib.util.spec_from_file_location(
    "critical_loads", "../../notebooks/critical_loads.py"
)
cl = importlib.util.module_from_spec(spec)
spec.loader.exec_module(cl)

In [3]:
# Connect to PostGIS
eng = nivapy.da.connect_postgis()

Connection successful.


# Høyanger: Calculate exceedances

**Note:** This notebook is rather messy and should be tidied up. In particular, cell 7 currently subsets the data to avoid issues with overlapping catchment polygons: the three non-overlapping polygons are processed simultaneously, and the fourth is then processed separately. This **requires running the notebook twice** (once for each catchment "subset"), each time changing the name of the summary CSV generated in section 5 to avoid overwriting. This is "hacky" and can be cleaned up, but it seems pragmatic for the moment.

## 1. Get catchments of interest

### 1.1. Outflow points

In [4]:
# Get outflows
sql = (
    "SELECT * FROM niva.stations "
    "WHERE station_id IN ( "
    "  SELECT station_id FROM niva.projects_stations "
    "  WHERE project_id IN ( "
    "    SELECT project_id FROM niva.projects "
    "    WHERE project_name = 'Høyanger' "
    "    ) "
    "  ) "
)
stn_gdf = gpd.read_postgis(sql, eng)

# Reproject to ETRS89 UTM Z33N
stn_gdf = stn_gdf.to_crs("epsg:25833")

stn_gdf.head()

,station_id,station_code,station_name,aquamonitor_id,longitude,latitude,geom
0,1285,EIR,Eiriksdal,None,6.215740,61.233612,POINT (29385.372 6820523.378)
1,1286,GAU,Gautingdalselva,None,6.144167,61.241150,POINT (25681.326 6821876.913)
2,1287,HAA,Haaland,None,6.077150,61.220030,POINT (21785.963 6820029.835)
3,1288,HOY,Hoyanger,None,6.073730,61.216397,POINT (21548.133 6819652.935)


### 1.2. Catchment boundaries

In [5]:
# Get catchments
stn_list = list(stn_gdf["station_id"].astype(str))
bind_pars = ",".join(stn_list)
sql = f"SELECT * FROM niva.catchments " f"WHERE station_id IN ({bind_pars})"
cat_gdf = gpd.read_postgis(sql, eng)

# Reproject to ETRS89 UTM Z33N
cat_gdf = cat_gdf.to_crs("epsg:25833")

# Join codes
cat_gdf = cat_gdf.merge(
    stn_gdf[["station_id", "station_code"]], how="left", on="station_id"
)
cat_gdf = cat_gdf[["station_id", "station_code", "geom"]]

# Add area
cat_gdf["cat_area_km2"] = cat_gdf["geom"].area / 1e6

cat_gdf.head()

,station_id,station_code,geom,cat_area_km2
0,1285,EIR,"MULTIPOLYGON (((37248.834 6823946.164, 37499.3...",71.965714
1,1286,GAU,"MULTIPOLYGON (((29139.510 6824275.830, 29235.2...",5.869828
2,1287,HAA,"MULTIPOLYGON (((22172.792 6822076.278, 22182.1...",3.608488
3,1288,HOY,"MULTIPOLYGON (((29139.510 6824275.830, 29235.2...",96.946486


## 2. Calculate critical loads

### 2.1. Process input template

Kari has supplied two input templates - one for the "main" catchments and one for the subcatchments. This avoid issues of overlapping polygons during the rasterisation steps. **This notebook should therefore be run twice, modifying `catch_set` below, as necessary**.


We wish to use the "F-factor" method for these calculations.

In [6]:
# Choose catchments to process (one of 'main', 'sub', 'sub_jan06')
catch_set = "main"

assert catch_set in ["sub", "main"]

In [7]:
# Path to completed template
xl_path = f"../data/input_template_critical_loads_water_v1-1_hoyanger_{catch_set}.xlsx"

# Read template
req_df = pd.read_excel(xl_path, sheet_name="required_parameters")
mag_df = pd.read_excel(xl_path, sheet_name="magic_parameters")
opt_df = pd.read_excel(xl_path, sheet_name="optional_parameters")

# Read quantites calculated in notebook 2 and patch input template
# Required pars
nupt_df = pd.read_csv(r"../output/hoyanger_nupt_summary.csv")
nupt_df = pd.merge(nupt_df, stn_gdf, how="left", on="station_id")
nupt_df = nupt_df[["station_name", "nupt_meqpm2pyr"]]
nupt_df.columns = ["Region_id", "Nupt"]
del req_df["Nupt"]
req_df = pd.merge(req_df, nupt_df, how="left", on="Region_id")

# Optional pars. See Section 3 here:
# https://github.com/JamesSample/critical_loads_2/blob/master/notebooks/workflow_update_2023/02d_blr_summaries.ipynb
lc_df = pd.read_csv(r"../output/hoyanger_ar50_summary.csv")
class_dict = {
    81: "Lake_area",
    30: "Forest_area",
    51: "Bare_area",
    70: "Bare_area",
    60: "Peat_area",
    -1: "Other_area",
}
lc_df["lc_class"] = np.where(lc_df["arveget"] == 51, lc_df["arveget"], lc_df["artype"])
lc_df["lc_class"] = np.where(
    lc_df["lc_class"].isin(class_dict.keys()), lc_df["lc_class"], -1
)
lc_df["lc_class"].replace(class_dict, inplace=True)
lc_df = lc_df.query(
    "lc_class in ('Lake_area', 'Forest_area', 'Bare_area', 'Peat_area')"
)
lc_df = lc_df[["station_id", "lc_class", "area_km2"]]
lc_df = lc_df.groupby(["station_id", "lc_class"]).sum()
lc_df = lc_df.unstack("lc_class").fillna(0)
lc_df.columns = lc_df.columns.get_level_values(1)
lc_df.columns.name = ""
lc_df.reset_index(inplace=True)

lc_df = pd.merge(lc_df, stn_gdf, how="left", on="station_id")
lc_df = pd.merge(
    lc_df, cat_gdf[["station_id", "cat_area_km2"]], how="left", on="station_id"
)
lc_df = lc_df[
    [
        "station_name",
        "cat_area_km2",
        "Lake_area",
        "Forest_area",
        "Bare_area",
        "Peat_area",
    ]
]
lc_df.columns = [
    "Region_id",
    "Catch_area",
    "Lake_area",
    "Forest_area",
    "Bare_area",
    "Peat_area",
]
opt_df.drop(
    ["Catch_area", "Lake_area", "Forest_area", "Bare_area", "Peat_area"],
    axis="columns",
    inplace=True,
)
opt_df = pd.merge(opt_df, lc_df, how="left", on="Region_id")

In [8]:
# Set BC0 method
bc0_method = "_Ffac"

# Calculate CLs
cl_df = cl.calculate_critical_loads_for_water(
    req_df=req_df, mag_df=mag_df, opt_df=opt_df
)

# Get cols of interest
cols = [
    "Region_id",
    f"CLAOAA{bc0_method}_meq/m2/yr",
    "ENO3_flux_meq/m2/yr",
    "CLminN_meq/m2/yr",
    f"CLmaxNoaa{bc0_method}_meq/m2/yr",
    f"CLmaxSoaa{bc0_method}_meq/m2/yr",
]
cl_df = cl_df[cols]

cl_df

/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Region_id,CLAOAA_Ffac_meq/m2/yr,ENO3_flux_meq/m2/yr,CLminN_meq/m2/yr,CLmaxNoaa_Ffac_meq/m2/yr,CLmaxSoaa_Ffac_meq/m2/yr
0,Hoyanger,61.182348,14.554914,1.632134,78.778131,62.231886


## 2.2. Rasterise critical loads

In [9]:
# Cell size for rasterisation
cell_size = 50

# Snap tiff
snap_tif = f"../raster/hoyanger_snap_ras_{cell_size}m.tif"

# Simplify col names (as units are consistent)
cl_df.columns = [i.split("_")[0].lower() for i in cl_df.columns]
cl_df.rename({"region": "station_name"}, inplace=True, axis=1)
cl_df.dropna(how="any", inplace=True)

# Add CLminS as 0
cl_df["clmins"] = 0

# Join to catchments
cl_df = pd.merge(
    cl_df, stn_gdf[["station_name", "station_code"]], on="station_name", how="left"
)
del cl_df["station_name"]
cat_gdf = cat_gdf.merge(cl_df, how="left", on="station_code")
cat_gdf.dropna(inplace=True)
cat_gdf.reset_index(inplace=True, drop=True)

# Save temporary file
temp = "../raster/temp.geojson"
cat_gdf.to_file(temp, driver="GeoJSON")

# Rasterize each column
cols = ["claoaa", "eno3", "clminn", "clmaxnoaa", "clmaxsoaa", "clmins"]
for col in cols:
    print(f"Rasterising {col}...")
    # Tiff to create
    out_tif = f"../raster/critical_loads/{col}_meqpm2pyr_{cell_size}m.tif"
    cl.vec_to_ras(temp, out_tif, snap_tif, col, -9999, "Float32")

# Delete temp file
os.remove(temp)

cat_gdf

Rasterising claoaa...
Rasterising eno3...
Rasterising clminn...
Rasterising clmaxnoaa...
Rasterising clmaxsoaa...
Rasterising clmins...


,station_id,station_code,geom,cat_area_km2,claoaa,eno3,clminn,clmaxnoaa,clmaxsoaa,clmins
0,1288,HOY,"MULTIPOLYGON (((29139.510 6824275.830, 29235.2...",96.946486,61.182348,14.554914,1.632134,78.778131,62.231886,0.0


## 3. Process deposition data

We will use the NILU historic series for 2012-16 (new method) to represent present-day conditions. For the future, we will use the EMEP scenarios for 2030 and 2050 provided for the GP review. A compatible EMEP run for 2015 (also provided for the GP review) will be used to calculate change factors that can be applied to the NILU series.

### 3.1. Select NILU deposition series

In [10]:
# List available series
with pd.option_context("display.max_colwidth", -1):
    ser_grid = cl.view_dep_series(eng).query("series_id in [27, 67]")
    display(ser_grid)

,series_id,name,short_name,grid,description
26,27,Middel 2012-2016 (new),1216_blrgrid,blr,Fordelt til BLR av NILU 2017 (Wenche Aas; new method)
66,67,Middel 2017-2021 (new),1721_blrgrid,blr,Supplied by Met in 2023 (Lewis Blake; new method)


We are interested in series IDs 27 and 67.

### 3.2. Rasterise NILU deposition data

In [11]:
# BLR grid, new method
ser_dict = {"1216": 27, "1721": 67}

for par in ["nitrogen", "sulphur"]:
    for period, ser_id in ser_dict.items():
        print(f"Rasterising {par}, {period}...")

        # Get dep data
        dep_gdf = cl.extract_deposition_as_gdf(
            ser_id, par, eng, veg_class="grid average"
        ).to_crs("epsg:25833")

        # Save temporary file
        temp = "../raster/temp.geojson"
        dep_gdf.to_file(temp, driver="GeoJSON")

        # Convert to raster
        col_name = f"{par[0]}dep_meqpm2pyr"
        out_tif = (
            f"../raster/deposition/{par[0]}dep_{period}_meqpm2pyr_{cell_size}m.tif"
        )
        cl.vec_to_ras(temp, out_tif, snap_tif, col_name, -9999, "Float32")

        # Delete temp file
        os.remove(temp)

Rasterising nitrogen, 1216...
Rasterising nitrogen, 1721...
Rasterising sulphur, 1216...
Rasterising sulphur, 1721...


### 3.3. Future EMEP scenarios

In [12]:
def calculate_dep_totals(ds):
    """Takes an xarray netCDF dataset in the format used by EMEP and calculates
    grid average deposition totals:

        Total SOx = Wet SOx + Dry SOx
        Total OxN = Wet OxN + Dry OxN
        Total RedN = Wet RedN + Dry RedN
        Total N = Total OxN + Total RedN

    Args:
        ds: xarray dataset

    Returns:
        New variables are added to 'ds'.
    """
    # Pars of interest
    par_list = [
        "DDEP_SOX_m2Grid",
        "WDEP_SOX",
        "DDEP_OXN_m2Grid",
        "WDEP_OXN",
        "DDEP_RDN_m2Grid",
        "WDEP_RDN",
    ]

    # Check N units are consistent
    for par in par_list:
        unit = ds[par].attrs["units"]
        assert unit in ["mgS/m2", "mgN/m2"], "Units not consistent."

    # Calculate total oxidised S
    ds["DEP_SOX"] = ds["WDEP_SOX"] + ds["DDEP_SOX_m2Grid"]
    ds["DEP_SOX"].attrs["units"] = "mgS/m2"

    # Calculate total oxidised N
    ds["DEP_OXN"] = ds["WDEP_OXN"] + ds["DDEP_OXN_m2Grid"]
    ds["DEP_OXN"].attrs["units"] = "mgN/m2"

    # Calculate total reduced N
    ds["DEP_RDN"] = ds["WDEP_RDN"] + ds["DDEP_RDN_m2Grid"]
    ds["DEP_RDN"].attrs["units"] = "mgN/m2"

    # Calculate total N
    ds["DEP_TOTN"] = (
        ds["WDEP_OXN"] + ds["WDEP_RDN"] + ds["DDEP_OXN_m2Grid"] + ds["DDEP_RDN_m2Grid"]
    )
    ds["DEP_TOTN"].attrs["units"] = "mgN/m2"

    return ds

In [13]:
# Read the scenario data and reproject to match the Høyanger snap grid
scenario = "Baseline"  # 'Baseline', 'MFR' or 'Diet_low'
snap_ds = rio.open_rasterio(snap_tif)
ds_dict = {}
for year in [2015, 2030, 2050]:
    fpath = f"/home/jovyan/shared/common/icp_waters/deposition_data/gp_review_scenarios/EMEP01_rv4.45_met2015_emis{year}_{scenario}_v1C_CCE-ICPwaters.nc"
    ds = rio.open_rasterio(fpath, mask_and_scale=True)
    ds.rio.write_crs(4326, inplace=True)
    ds = calculate_dep_totals(ds)
    ds = ds.rio.reproject_match(snap_ds, resampling=Resampling.bilinear)
    ds_dict[year] = ds

In [14]:
# Calculate change factors from EMEP as (future/2015_baseline), then apply
# these factors to the NILU 2012-16 data to estimate future deposition
nilu_series_name = "1216"
base_ds = ds_dict[2015].copy()
for year in [2030, 2050]:
    fut_ds = ds_dict[year].copy()
    for par in ["DEP_TOTN", "DEP_SOX"]:
        fac_ds = fut_ds[par] / base_ds[par]

        print(year, par, fac_ds.squeeze().min().values, fac_ds.squeeze().max().values)

        if par.endswith("N"):
            nilu_path = f"../raster/deposition/ndep_{nilu_series_name}_meqpm2pyr_{cell_size}m.tif"
            out_tif = f"../raster/deposition/ndep_{year}bc_meqpm2pyr_{cell_size}m.tif"
        else:
            nilu_path = f"../raster/deposition/sdep_{nilu_series_name}_meqpm2pyr_{cell_size}m.tif"
            out_tif = f"../raster/deposition/sdep_{year}bc_meqpm2pyr_{cell_size}m.tif"

        nilu_ds = rio.open_rasterio(nilu_path, mask_and_scale=True)
        bias_cor_ds = nilu_ds.squeeze() * fac_ds.squeeze()
        bias_cor_ds.rio.to_raster(out_tif)

2030 DEP_TOTN 0.7238527536392212 0.7586063146591187
2030 DEP_SOX 0.8607051372528076 0.9165170788764954
2050 DEP_TOTN 0.62989741563797 0.6868270635604858
2050 DEP_SOX 0.8461339473724365 0.9073213934898376


## 4. Calculate exceedances

### 4.1. SSWC

**Note:** Values <0 are no longer set back to zero in the code below. See e-mail from Kari received 11.06.2020 at 15.08. Uncomment the line in the cell below to use the "standard" approach.

In [15]:
for period in ["1216", "1721", "2030bc", "2050bc"]:
    # Read grids
    s_tif = f"../raster/deposition/sdep_{period}_meqpm2pyr_{cell_size}m.tif"
    s_dep, s_ndv, epsg, extent = nivapy.spatial.read_raster(s_tif)

    eno3_tif = f"../raster/critical_loads/eno3_meqpm2pyr_{cell_size}m.tif"
    eno3fl, eno3_ndv, epsg, extent = nivapy.spatial.read_raster(eno3_tif)

    claoaa_tif = f"../raster/critical_loads/claoaa_meqpm2pyr_{cell_size}m.tif"
    claoaa, cla_ndv, epsg, extent = nivapy.spatial.read_raster(claoaa_tif)

    # Set ndv
    s_dep[s_dep == s_ndv] = np.nan
    eno3fl[eno3fl == eno3_ndv] = np.nan
    claoaa[claoaa == cla_ndv] = np.nan

    # Set negative to zero
    claoaa[claoaa < 0] = 0

    # Exceedance
    sswc_ex = s_dep + eno3fl - claoaa
    del s_dep, eno3fl, claoaa

    # Set <0 to 0
    # sswc_ex[sswc_ex < 0] = 0

    # Write geotif
    sswc_tif = f"../raster/exceedance/sswc_ex_{period}_meqpm2pyr_{cell_size}m.tif"
    cl.write_geotiff(sswc_ex, sswc_tif, snap_tif, -1, gdal.GDT_Float32)
    del sswc_ex

### 4.2. FAB

In [16]:
# Read CL arrays
for period in ["1216", "1721", "2030bc", "2050bc"]:
    array_dict = {}

    for name in ["clminn", "clmaxnoaa", "clmins", "clmaxsoaa"]:
        # Read tif
        tif_path = f"../raster/critical_loads/{name}_meqpm2pyr_{cell_size}m.tif"
        data, ndv, epsg, extent = nivapy.spatial.read_raster(tif_path)
        data[data == ndv] = np.nan
        array_dict[name] = data

    # Read dep arrays
    for name in ["ndep", "sdep"]:
        # Read tif
        tif_path = f"../raster/deposition/{name}_{period}_meqpm2pyr_{cell_size}m.tif"
        data, ndv, epsg, extent = nivapy.spatial.read_raster(tif_path)
        data[data == ndv] = np.nan
        array_dict[name] = data

    # Extract arrays from dict
    cln_min = array_dict["clminn"]
    cln_max = array_dict["clmaxnoaa"]
    cls_min = array_dict["clmins"]
    cls_max = array_dict["clmaxsoaa"]
    dep_n = array_dict[f"ndep"]
    dep_s = array_dict[f"sdep"]

    # Estimate exceedances
    ex_n, ex_s, reg_id = cl.vectorised_exceed_ns_icpm(
        cln_min, cln_max, cls_min, cls_max, dep_n, dep_s
    )

    # Save GeoTiffs
    # N
    n_tif = f"../raster/exceedance/fab_ex_n_{period}_meqpm2pyr_{cell_size}m.tif"
    cl.write_geotiff(ex_n, n_tif, snap_tif, -1, gdal.GDT_Float32)

    # S
    s_tif = f"../raster/exceedance/fab_ex_s_{period}_meqpm2pyr_{cell_size}m.tif"
    cl.write_geotiff(ex_s, s_tif, snap_tif, -1, gdal.GDT_Float32)

    # N+S
    ns_tif = f"../raster/exceedance/fab_ex_ns_{period}_meqpm2pyr_{cell_size}m.tif"
    cl.write_geotiff(ex_n + ex_s, ns_tif, snap_tif, -1, gdal.GDT_Float32)

    # Exceedance 'region'
    reg_tif = f"../raster/exceedance/fab_ex_reg_id_{period}_{cell_size}m.tif"
    cl.write_geotiff(reg_id, reg_tif, snap_tif, -1, gdal.GDT_Float32)

## 5. Summary statistics

In [17]:
# Get paths to all dep and ex grids
search_path1 = "../raster/exceedance/*.tif"
flist1 = glob.glob(search_path1)
search_path2 = "../raster/deposition/*.tif"
flist2 = glob.glob(search_path2)
search_path3 = "../raster/critical_loads/*.tif"
flist3 = glob.glob(search_path3)
flist = flist1 + flist2 + flist3

df_list = []
for fname in flist:
    ds_name = os.path.split(fname)[1][:-4]
    sum_df = nivapy.spatial.zonal_statistics(
        cat_gdf[["station_id", "station_code", "geom"]], fname, ""
    )
    sum_df["dataset"] = ds_name
    df_list.append(sum_df)

sum_df = pd.concat(df_list, sort=True)
sum_df = sum_df[
    ["dataset", "station_code", "min_", "mean_", "max_", "std_", "count_"]
].reset_index(drop=True)

# Save
csv_path = f"../output/hoyanger_results_summary_meqpm2pyr_{catch_set}_catches.csv"
sum_df.to_csv(csv_path, index=False)

sum_df.head(20)

/opt/conda/lib/python3.10/site-packages/rasterstats/io.py:328: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


,dataset,station_code,min_,mean_,max_,std_,count_
0,sswc_ex_1721_meqpm2pyr_50m,HOY,-38.080956,-37.833580,-36.833298,0.420881,38743
1,fab_ex_reg_id_2050bc_50m,HOY,0.000000,0.002244,3.000000,0.082016,38772
2,fab_ex_n_1721_meqpm2pyr_50m,HOY,0.000000,0.000000,0.000000,0.000000,38743
3,fab_ex_reg_id_1721_50m,HOY,0.000000,0.002244,3.000000,0.082016,38772
4,fab_ex_ns_2050bc_meqpm2pyr_50m,HOY,0.000000,0.000000,0.000000,0.000000,38743
5,fab_ex_s_1721_meqpm2pyr_50m,HOY,0.000000,0.000000,0.000000,0.000000,38743
6,fab_ex_n_2030bc_meqpm2pyr_50m,HOY,0.000000,0.000000,0.000000,0.000000,38743
7,sswc_ex_2030bc_meqpm2pyr_50m,HOY,-38.519913,-37.460735,-35.478287,0.962749,38743
8,fab_ex_s_2030bc_meqpm2pyr_50m,HOY,0.000000,0.000000,0.000000,0.000000,38743
9,fab_ex_n_2050bc_meqpm2pyr_50m,HOY,0.000000,0.000000,0.000000,0.000000,38743
